<a href="https://colab.research.google.com/github/Varun-Mulchandani/roBERTa_based_SQuAD_QA/blob/master/roBERTa_SQuAD_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Confirming GPU

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

For this project, I will be using HuggingFace's transformers library

In [0]:
!pip install transformers

Import necessary libraries

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
print('TF version', tf.__version__)

TF version 2.2.0-rc4


Max length for the input sequence is reduced to 384 due to lack of computaitonal resources for now.

In [0]:
MAX_LEN = 384

tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file = 'drive/My Drive/vocab-roberta-base.json',
    merges_file = 'drive/My Drive/merges-roberta-base.txt',
    lowercase = True,
    add_prefix_space = True
)

Loading the SQuAD dataset which has been converted to a csv file:

In [0]:
train = pd.read_csv('drive/My Drive/train (3).csv').fillna('')
train.head()

,Unnamed: 0,title,question,id,answers,answer_start,context
0,0,Beyoncé,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,in the late 1990s,269,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,1,Beyoncé,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,singing and dancing,207,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,2,Beyoncé,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,2003,526,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,3,Beyoncé,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,"Houston, Texas",166,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4,4,Beyoncé,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...


In [0]:
rec = train.shape[0]  # Number of records in the training set
inputs = np.ones((rec, MAX_LEN), dtype = 'int32') # Input vector
attention_mask = np.zeros((rec, MAX_LEN), dtype = 'int32') # Attention Mask
token_type_ids = np.zeros((rec, MAX_LEN), dtype = 'int32') # Tokens produced
start_tokens = np.zeros((rec, MAX_LEN), dtype = 'int32') # Start logit for answer
end_tokens = np.zeros((rec, MAX_LEN), dtype = 'int32') # End logit for answer

for i in range(rec):

  context = ' '+' '.join(train.loc[i, 'context'].split())
  answer = ' '+' '.join(train.loc[i, 'answers'].split())
  question = ' '+' '.join(train.loc[i, 'question'].split())

  start_idx = train.loc[i, 'answer_start']

  chars = np.zeros((len(context)))
  chars[start_idx:start_idx + len(answer)] = 1
  if context[start_idx - 1] == ' ':
    chars[start_idx - 1] = 1
  
  enc1 = tokenizer.encode(context)
  enc2 = tokenizer.encode(question)

  # For resource limitations only.

  if len(enc1) + len(enc2) + 4 < MAX_LEN:

    #creating offsets
    offsets = []
    start_idx = 0

    for t in enc1.ids:
      w = tokenizer.decode([t])
      offsets.append((start_idx, start_idx + len(w)))
      start_idx += len(w)
    
    tokens = []
    for j, (a, b) in enumerate(offsets):
      sum_ = np.sum(chars[a:b])
      if sum_ > 0:
        tokens.append(j)

    # The input for roberta is in the form <s> Question </s></s> Answer </s>
    
    inputs[i, :len(enc1.ids) + len(enc2.ids) + 4] = [0] + enc2.ids + [2,2] + enc1.ids + [2]

    attention_mask[i, :len(enc1.ids) + len(enc2.ids) + 4] = 1

    if len(tokens) > 0:
      start_tokens[i, tokens[0] + 1] = 1
      end_tokens[i, tokens[-1] + 1] = 1

In [0]:
def build_model():
  ids = tf.keras.layers.Input((MAX_LEN,), dtype = tf.int32)
  att = tf.keras.layers.Input((MAX_LEN,), dtype = tf.int32)
  tok = tf.keras.layers.Input((MAX_LEN,), dtype = tf.int32)

  config = RobertaConfig.from_pretrained('drive/My Drive/config-roberta-base.json')
  bert_model = TFRobertaModel.from_pretrained('drive/My Drive/pretrained-roberta-base.h5', config = config)
  x = bert_model(ids, attention_mask=att, token_type_ids = tok)

  # For start logit

  x1 = tf.keras.layers.Dropout(0.1)(x[0])
  x1 = tf.keras.layers.Conv1D(1,1)(x1)
  x1 = tf.keras.layers.Flatten()(x1)
  x1 = tf.keras.layers.Activation('softmax')(x1)

  # For end logit

  x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
  x2 = tf.keras.layers.Conv1D(1,1)(x2)
  x2 = tf.keras.layers.Flatten()(x2)
  x2 = tf.keras.layers.Activation('softmax')(x2)

  # Initalising the model

  model = tf.keras.models.Model(inputs = [ids, att, tok], outputs = [x1, x2])
  optimizer = tf.keras.optimizers.Adam(learning_rate = 3e-5)
  model.compile(loss='categorical_crossentropy', optimizer = optimizer)

  return model

In [0]:
model = build_model()

In [0]:
history = model.fit([inputs, attention_mask, token_type_ids], [start_tokens, end_tokens], epochs = 3, batch_size = 4, validation_split = 0.1)

Epoch 1/3
19535/19535 [==============================] - 4760s 244ms/step - loss: 5.7435 - activation_7_loss: 2.8490 - activation_8_loss: 2.8946 - val_loss: 3.8448 - val_activation_7_loss: 1.9756 - val_activation_8_loss: 1.8692
Epoch 2/3
19535/19535 [==============================] - 4756s 243ms/step - loss: 3.5832 - activation_7_loss: 1.8125 - activation_8_loss: 1.7707 - val_loss: 3.1420 - val_activation_7_loss: 1.5986 - val_activation_8_loss: 1.5434
Epoch 3/3
19535/19535 [==============================] - 4747s 243ms/step - loss: 2.8444 - activation_7_loss: 1.4487 - activation_8_loss: 1.3957 - val_loss: 2.7097 - val_activation_7_loss: 1.4078 - val_activation_8_loss: 1.3019
